In [ ]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd
import numpy as np
import math
import os
import unicodedata
import string
import time
import ast
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
thread_one = pd.read_csv("../../data/thread_one.csv")
thread_two = pd.read_csv("../../data/thread_two.csv")
df = pd.DataFrame()
df = df.append(thread_one, ignore_index=True)
df = df.append(thread_two, ignore_index=True)
df = df.drop(df.columns[0], axis=1)


In [ ]:
c_anchors = []
members = []
d_anchors =[]
directors_list=[]

for row in df.itertuples():
  if type(row.cast_members)==str:
    cast = row.cast_members
    cast = ast.literal_eval(cast)
    members.append(cast)
  else:
    members.append(np.nan)
  
  if type(row.cast_anchors)==str:
    anchors = row.cast_anchors
    anchors = ast.literal_eval(anchors)
    c_anchors.append(anchors)
  else:
    c_anchors.append(np.nan)
  
  if type(row.directors)==str:
    directors = row.directors
    directors = ast.literal_eval(directors)
    new_directors = []
    for director in directors:
      new_directors.append(director.strip())
    directors = new_directors
    directors_list.append(directors)
  else:
    directors_list.append(np.nan)
  
  if type(row.director_anchors)==str:
    anchors = ast.literal_eval(row.director_anchors)
    d_anchors.append(anchors)
  else:
    d_anchors.append(np.nan)
df['cast_anchors'] = c_anchors
df['cast_members'] = members
df['directors'] = directors_list
df['director_anchors'] = d_anchors

In [ ]:
# pd.set_option('display.max_columns', 500)
# df.head()

In [ ]:
def retry(ExceptionToCheck, tries=20, delay=3, backoff=2, logger=None):
  """
    Modified from source: http://wiki.python.org/moin/PythonDecoratorLibrary#Retry

    Objective
    ----------
    Exponential backoff function.

    Parameters
    ----------
    ExceptionToCheck : Exception or tuple
      the exception to check. may be a tuple of exceptions to check
      Possible values:
        ConnectionResetError, (TimeoutError, ConnectionError)
    tries : int
      number of times to try (not retry) before giving up
    delay : int
      initial delay between retries in seconds
    backoff: int
      backoff multiplier
      E.g. value of 2 will double the delay each retry
    logger : logging.Logger instance
      logger to use
  """
  def deco_retry(f):
    @wraps(f)
    def f_retry(*args, **kwargs):
      mtries, mdelay = tries, delay
      while mtries > 1:
        try:
          return f(*args, **kwargs)
        except ExceptionToCheck:
          msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
          if logger:
            #logger.exception(msg) # would print stack trace
            logger.warning(msg)
          else:
            print(msg)
          time.sleep(mdelay)
          mtries -= 1
          mdelay *= backoff
        return f(*args, **kwargs)
      return f_retry  # true decorator
  return deco_retry  

In [ ]:
cast = pd.DataFrame()

for row in df.itertuples():
  if type(row.cast_members)==list:
    first_billed = row.cast_members[:5]
    first_anchors = row.cast_anchors[:5]
    for idx, actor in enumerate(first_billed):
      cast_member = first_billed[idx]
      link = first_anchors[idx]
      temp = pd.DataFrame({
        'actor': [cast_member],
        'link': [link]
      })
      cast = cast.append(temp, ignore_index=True)
  else:
    temp = pd.DataFrame({
      'actor': [np.nan],
      'link': [np.nan]
    })
    cast = cast.append(temp, ignore_index=True)
    
cast.head()
print(f"Length Cast:{len(cast)}")
cast = cast.drop_duplicates(subset='actor', keep='first')
cast = cast.dropna()
print(f"Length Cast:{len(cast)}")
      

In [2]:
import pandas as pd
df = pd.read_csv("../cast_oscars_1.csv")

In [3]:
df.describe()

,Unnamed: 0,year
count,7936.000000,793.000000
mean,3967.500000,1995.030265
std,2291.070201,18.441461
min,0.000000,1931.000000
25%,1983.750000,1984.000000
50%,3967.500000,1999.000000
75%,5951.250000,2010.000000
max,7935.000000,2020.000000


In [7]:
df.loc[(df['actor']=="Will Smith")]

,Unnamed: 0,actor,year,outcome,award,movie,movie_year
700,700,Will Smith,NaN,NaN,NaN,NaN,NaN


In [ ]:
#SPLIT INTO MULTIPLE THREADS
subset = cast[:10276]
cast = subset
cast = cast[329:]

In [ ]:
def grabAverage(actor, link):
  try:
    options = webdriver.ChromeOptions()
    options.add_argument("no-sandbox")
    options.add_argument("disable-dev-shm-usage")
    options.add_argument("headless")
    user_data_path = "INSERT HERE"
    driver_path = "INSERT HERE"
    options.add_argument(f"user-data-dir=C:/Users/{user_data_path}")
    driver = webdriver.Chrome(f"{driver_path}", chrome_options=options)

    @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
    def serve_link():
      driver.get(f"https://pro.imdb.com{link}boxoffice")
    serve_link()
    try:
      if driver.find_element_by_id('a-autoid-4').text == 'Actor':
        sourcez = driver.page_source
        cast_html = BeautifulSoup(sourcez, 'html.parser')
        avg = cast_html.find('table', {'id': 'box_office_mojo'}).find('div', text=" US & Canada\n").parent.parent.find_all('td')[2].text.split("\n")[0]
        temp = pd.DataFrame({
          'actor': [actor],
          'avg': [avg]
        })
        driver.quit()
#         print(Fore.GREEN + f"{director}: {avg}")
      else:
        driver.find_element_by_id('a-autoid-4-announce').click()
        driver.find_element_by_xpath("//a[contains(@class, 'a-dropdown-link') and contains(text(), 'Actor')]").click()
        driver.execute_script("return document.getElementById('box_office_mojo').remove()")
        table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "box_office_mojo")));
        sourcez = driver.page_source
        cast_html = BeautifulSoup(sourcez, 'html.parser')
        avg = cast_html.find('table', {'id': 'box_office_mojo'}).find('div', text=" US & Canada\n").parent.parent.find_all('td')[2].text.split("\n")[0]
        temp = pd.DataFrame({
          'actor': [actor],
          'avg': [avg]
        })
        driver.quit()
#         print(Fore.GREEN + f"{director}: {avg}")
    except:
      sourcez = driver.page_source
      cast_html = BeautifulSoup(sourcez, 'html.parser')
      avg = cast_html.find('table', {'id': 'box_office_mojo'}).find('div', text=" US & Canada\n").parent.parent.find_all('td')[2].text.split("\n")[0]
      temp = pd.DataFrame({
        'actor': [actor],
        'avg': [avg]
      })
      driver.quit()
#       print(Fore.GREEN + f"{director}: {avg}")


  except Exception as e:
    print(e)
    avg = np.nan
    temp = pd.DataFrame({
      'actor': [actor],
      'avg': [avg]
    })
    
    
  print(Fore.GREEN + f"{actor}: {avg}")
  print(Fore.GREEN + f"{counter}/{len(cast)}")
  clear_output(wait=True)
  return temp

In [ ]:
'''
  READ CSV
'''
cast_averages = pd.read_csv("./actor_avg_one.csv")
cast_averages = cast_averages[:329]
print(len(cast_averages))

In [ ]:
'''
  GRAB CAST AVERAGES
'''

# cast_averages = pd.DataFrame()
counter = 0
for row in cast.itertuples():
  counter+=1
  link = row.link
  actor = row.actor
  info = grabAverage(actor, link)
  cast_averages = cast_averages.append(info, ignore_index=True)
  cast_averages.to_csv("./actor_avg_one.csv")
print("Averages One Complete")